# Introduction

I will explore the problem in following stages:

1.  **Hypothesis Generation** – understanding the problem better by brainstorming possible factors that can impact the outcome
2.  **Data Exploration** – looking at categorical and continuous feature summaries and making inferences about the data.
3.  **Data Cleaning** – imputing missing values in the data and checking for outliers
4.  **Feature Engineering** – modifying existing variables and creating new ones for analysis
5.  **Model Building** – making predictive models on the data


## 1. Hypothesis Generation

This is a very pivotal step in the process of analyzing data. This involves understanding the problem and making some hypothesis about what could potentially have a good impact on the outcome. This is done BEFORE looking at the data, and we end up creating a laundry list of the different analysis which we can potentially perform if data is available.

### The Problem Statement

Understanding the problem statement is the first and foremost step:

> In this competition, you will forecast the demand of a product for a given week, at a particular store. The dataset you are given consists of 9 weeks of sales transactions in Mexico. Every week, there are delivery trucks that deliver products to the vendors. Each transaction consists of sales and returns. Returns are the products that are unsold and expired. The demand for a product in a certain week is defined as the sales this week subtracted by the return next week.

So the idea is to find out the demand of a product (sales - returns) per client, and store which impacts the sales of a product. Let’s think about some of the analysis that can be done and come up with certain hypothesis.

### The Hypotheses

I came up with the following hypothesis while thinking about the problem. Since we’re talking about stores and products, lets make different sets for each.

**Store/Client Level Hypotheses:**

1.  **Town type:** Stores located in urban or Tier 1 towns should have higher sales because of the higher income levels of people there.
2.  **Population Density:** Stores located in densely populated areas should have higher sales because of more demand.
3.  **Store Capacity:** Stores which are very big in size should have higher sales as they act like one-stop-shops and people would prefer getting everything from one place
4.  **Competitors:** Stores having similar establishments nearby should have less sales because of more competition.
5.  **Marketing:** Stores which have a good marketing division should have higher sales as it will be able to attract customers through the right offers and advertising.
6.  **Location:** Stores located within popular marketplaces should have higher sales because of better access to customers.
7.  **Customer Behavior:** Stores keeping the right set of products to meet the local needs of customers will have higher sales.
8.  **Ambiance:** Stores which are well-maintained and managed by polite and humble people are expected to have higher footfall and thus higher sales.
9.  **Season:** Store should sell more after customer's pay day: after 15th or 30th of the month

**Product Level Hypotheses:**

1.  **Brand:** Branded products should have higher sales because of higher trust in the customer.
2.  **Packaging:** Products with good packaging can attract customers and sell more.
3.  **Utility:** Daily use products should have a higher tendency to sell as compared to the specific use products.
4.  **Display Area:** Products which are given bigger shelves in the store are likely to catch attention first and sell more.
5.  **Visibility in Store:** The location of product in a store will impact sales. Ones which are right at entrance will catch the eye of customer first rather than the ones in back.
6.  **Advertising:** Better advertising of products in the store will should higher sales in most cases.
7.  **Promotional Offers:** Products accompanied with attractive offers and discounts will sell more.


Lets move on to the data exploration where we will have a look at the data in detail.

## 2\. Data Exploration

We’ll be performing some basic data exploration here and come up with some inferences about the data.

The first step is to look at the data and try to identify the information which we hypothesized vs the available data. A comparison between the data dictionary on the competition page and out hypotheses is shown below:

![Image of Variables vs Hypothesis](files/../input-data/Variables_vs_Hyphotesis.png)

We can summarize the findings as:

** 9 Features Hypothesized but not found in actual data. **

** 5 Features Hypothesized as well as present in the data **

** 3 Features present in the data but not hypothesized. **


We invariable find features which we hypothesized, but data doesn’t carry and vice versa. We should look for open source data to fill the gaps if possible. Let’s start by loading the required libraries and data. 

In [1]:
import pandas as pd
import numpy as np
import time
import csv

_start_time = time.time()

def tic():
    global _start_time 
    _start_time = time.time()

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    print('Time passed: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))


In [66]:
#Read files:
tic()
train = pd.read_csv('input-data/train_sampled5pct.csv',
                           dtype  = {'Semana': 'int8',
                                     'Producto_ID':'int32',
                                     'Cliente_ID':'int32',
                                     'Agencia_ID':'uint16',
                                     'Canal_ID':'int8',
                                     'Ruta_SAK':'int32',
                                     'Venta_hoy':'float32',
                                     'Venta_uni_hoy': 'int32',
                                     'Dev_uni_proxima':'int32',
                                     'Dev_proxima':'float32',
                                     'Demanda_uni_equil':'float64'})
test = pd.read_csv('input-data/test.csv',
                           dtype  = {'Semana': 'int8',
                                     'Producto_ID':'int32',
                                     'Cliente_ID':'int32',
                                     'Agencia_ID':'uint16',
                                     'Canal_ID':'int8',
                                     'Ruta_SAK':'int32'})
tac()

Time passed: 0hour:0min:11sec


In [5]:
train.head()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0,4
2,3,1110,7,3301,15766,1238,4,39.320000,0,0,4
3,3,1110,7,3301,15766,1240,4,33.520000,0,0,4
4,3,1110,7,3301,15766,1242,3,22.920000,0,0,3


In [6]:
test.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277


In [67]:
#Since test dataframe is not the same as train dataframe, we make them equal by removing and adding columns
train.insert(0, 'id', np.nan)
test.insert(7, 'Venta_uni_hoy', np.nan)
test.insert(8, 'Venta_hoy', np.nan)
test.insert(9, 'Dev_uni_proxima', np.nan)
test.insert(10, 'Dev_proxima', np.nan)
test.insert(11, 'Demanda_uni_equil', np.nan)

In [8]:
train.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,NaN,3,1110,7,3301,15766,1212,3,25.139999,0,0,3
1,NaN,3,1110,7,3301,15766,1216,4,33.520000,0,0,4
2,NaN,3,1110,7,3301,15766,1238,4,39.320000,0,0,4
3,NaN,3,1110,7,3301,15766,1240,4,33.520000,0,0,4
4,NaN,3,1110,7,3301,15766,1242,3,22.920000,0,0,3


In [9]:
test.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,0,11,4037,1,2209,4639078,35305,NaN,NaN,NaN,NaN,NaN
1,1,11,2237,1,1226,4705135,1238,NaN,NaN,NaN,NaN,NaN
2,2,10,2045,1,2831,4549769,32940,NaN,NaN,NaN,NaN,NaN
3,3,11,1227,1,4448,4717855,43066,NaN,NaN,NaN,NaN,NaN
4,4,11,1219,1,1130,966351,1277,NaN,NaN,NaN,NaN,NaN


Its generally a good idea to combine both train and test data sets into one, perform feature engineering and then divide them later again. This saves the trouble of performing the same steps twice on test and train. Lets combine them into a dataframe ‘data’ with a ‘source’ column specifying where each observation belongs.

In [68]:
tic()
train['source']='train'
test['source']='test'
data = pd.concat([train, test],ignore_index=True)
tac()
print (train.shape, test.shape, data.shape)

Time passed: 0hour:0min:3sec
(3707724, 13) (6999251, 13) (10706975, 13)


Thus we can see that data has same #columns but rows equivalent to both test and train. Lets start by checking which columns contain missing values. (takes 32mins to run!)

In [9]:
data.apply(lambda x: sum(x.isnull()))

id                   74180464
Semana                      0
Agencia_ID                  0
Canal_ID                    0
Ruta_SAK                    0
Cliente_ID                  0
Producto_ID                 0
Venta_uni_hoy         6999251
Venta_hoy             6999251
Dev_uni_proxima       6999251
Dev_proxima           6999251
Demanda_uni_equil     6999251
source                      0
dtype: int64

There doesn't seem to be any missing values (other than the NaN we set on the test and train sets).

Lets look at some basic statistics for numerical variables.

In [11]:
data.describe()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
count,6999251.00000,81179715.000000,81179715.000000,81179715.000000,81179715.000000,8.117972e+07,81179715.000000,74180464.000000,74180464.000000,74180464.000000,74180464.000000,74180464.000000
mean,3499625.00000,6.341839,2533.745597,1.384793,2116.852025,1.803586e+06,20954.817836,7.310163,68.544523,0.130258,1.243248,7.224564
std,2020509.86883,2.313453,4069.579106,1.467666,1488.853067,2.406086e+06,18670.562503,21.967337,338.979516,29.323204,39.215523,21.771193
min,0.00000,3.000000,1110.000000,1.000000,1.000000,2.600000e+01,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1749812.50000,4.000000,1311.000000,1.000000,1161.000000,3.567150e+05,1242.000000,2.000000,16.760000,0.000000,0.000000,2.000000
50%,3499625.00000,6.000000,1613.000000,1.000000,1286.000000,1.193935e+06,30549.000000,3.000000,30.000000,0.000000,0.000000,3.000000
75%,5249437.50000,8.000000,2036.000000,1.000000,2803.000000,2.372538e+06,37569.000000,7.000000,56.099998,0.000000,0.000000,6.000000
max,6999250.00000,11.000000,25759.000000,11.000000,9991.000000,2.015152e+09,49997.000000,7200.000000,647360.000000,250000.000000,130760.000000,5000.000000


Some observations:

   Looking at Demanda_uni_equil (our target), or the amount of product sold per week, we find interesting things:
   
   **1)** The average is 7.22, so in average there is 7 units per week per store sold.
   
   **2)** Looking at the max of 5000, it looks very far fro the mean (3 orders of magnitude), so we must check for an outlier here or a store that is crazy different from the rest.
   
   **3)** Same behaviour we find on Dev_uni_proxima, Venta_hoy and Venta_uni_hoy
   
Looking at the nice data analysis made in R by Faviens, here: https://www.kaggle.com/fabienvs/grupo-bimbo-inventory-demand/notebook-8a62eda039a3b0b944cf/notebook we corroborate the outlier(s):
There is a massive client: Puebla Remision
   
![Image of size of Customers]( https://www.kaggle.io/svf/267812/783a24d1dd546819a44914f996b249e8/__results___files/figure-html/unnamed-chunk-16-1.png)
   

Moving to nominal (categorical) variable, lets have a look at the number of unique values in each of them.

In [11]:
data.apply(lambda x: len(x.unique()))

id                   6999252
Semana                     9
Agencia_ID               552
Canal_ID                   9
Ruta_SAK                3620
Cliente_ID            890267
Producto_ID             1833
Venta_uni_hoy           2117
Venta_hoy              73516
Dev_uni_proxima          559
Dev_proxima            14241
Demanda_uni_equil       2092
source                     2
dtype: int64

So, in train and test sets, we have 552 Agencies(depots), 890k clients (we might have some repeated clients due to typos when enterind data), 1833 products (we might have some repeated products here based on typos) and 3620 routes

## 3\. Data Cleaning

This step typically involves imputing missing values and treating outliers. As we saw before, there are no missing values. Regarding outliers, there seem to be an obvious one, but we are going to see later on if its necessary to treat it differently.

My initial reaction would be to see if anything with the word REMISION is on the test set. if not, then delete it. See this discussion: https://www.kaggle.com/c/grupo-bimbo-inventory-demand/forums/t/22037/puebla-remission/126053

In [ ]:
#Let's find out who are the clients with the word REMISION on it
client_name = pd.read_csv('files/../input-data/cliente_tabla.csv')
cliente_id_name_train = pd.merge(train,client_name, on='Cliente_ID')
cliente_id_name_test = pd.merge(test,client_name, on='Cliente_ID')

In [ ]:
cliente_id_name_train.head()

In [ ]:
cliente_id_name_train[cliente_id_name_train.NombreCliente.str.contains('REMISION')].count()

As we can see above, the word "REMISION" shows up 140k times on the train set. Let's see the test set:

In [ ]:
cliente_id_name_test[cliente_id_name_test.NombreCliente.str.contains('REMISION')].count()

12k rows shows up the word REMISION on the test set. This implies that it has to be predicted as well. We cannot eliminate it.

## 4\. Feature Engineering

We explored some nuances in the data in the data exploration section. Lets move on to resolving them and making our data ready for analysis. We will also create some new variables using the existing ones in this section.

### Feature1: Lag Features - Demand per client-product pair for prior weeks
Based on this blog: http://blog.nycdatascience.com/student-works/predicting-demand-historical-sales-data-grupo-bimbo-kaggle-competition/

As this script said: https://www.kaggle.com/bpavlyshenko/grupo-bimbo-inventory-demand/bimbo-xgboost-r-script-lb-0-457/code
It is important to know what were the previous weeks sales. If the previous week, too many products were supplied and they were not sold, the next week this product amount, supplied to the same store, will be decreased. So it is very important to included lag values of target variable as a feature to predict the next sales.

In [12]:
#log transformed mean
def log1p_mean (x):
    return np.expm1(np.mean(np.log1p(x)))

In [69]:
tic()
semana_cliente_producto_log1p_mean = data.groupby(['Semana','Cliente_ID','Producto_ID'],as_index=False).agg({'Demanda_uni_equil':  lambda x: log1p_mean(x)})
tac()

Time passed: 0hour:18min:17sec


In [70]:
#df_SCP_log = semana_cliente_producto_log1p_mean.copy()

In [73]:
semana_cliente_producto_log1p_mean.head()

,Semana,Cliente_ID,Producto_ID,Demanda_uni_equil
0,3,26,35148,15
1,3,26,43246,20
2,3,65,30007,130
3,3,65,36947,48
4,3,65,43173,84


In [77]:
print (data.dtypes)

id                   float64
Semana                  int8
Agencia_ID            uint16
Canal_ID                int8
Ruta_SAK               int32
Cliente_ID             int32
Producto_ID            int32
Venta_uni_hoy        float64
Venta_hoy            float64
Dev_uni_proxima      float64
Dev_proxima          float64
Demanda_uni_equil    float64
source                object
Log_Target_lag1      float64
Log_Target_lag2      float64
Log_Target_lag3      float64
Log_Target_lag4      float64
dtype: object


In [76]:
#here we add the number of lags we want
tic()
lag=4
for i in range(1,lag+1):
    semana_cliente_producto_log1p_mean['Semana'] += 1
    semana_cliente_producto_log1p_mean.rename(columns={semana_cliente_producto_log1p_mean.columns[3]: 'Log_Target_lag%d' %(i)}, inplace=True)
    data = pd.merge(data,semana_cliente_producto_log1p_mean, how = 'inner', on = ['Semana','Cliente_ID','Producto_ID'])
tac()

Time passed: 0hour:0min:33sec


In [16]:
data.tail()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,source,Log_Target_lag1,Log_Target_lag2,Log_Target_lag3,Log_Target_lag4
11719520,6999229,10,4051,1,1106,122768,43202,NaN,NaN,NaN,NaN,NaN,test,6,7,1,4
11719521,6999239,11,1427,1,2801,1373090,43307,NaN,NaN,NaN,NaN,NaN,test,NaN,2,8,8
11719522,6999242,10,1338,1,1201,789056,1146,NaN,NaN,NaN,NaN,NaN,test,1,1,2,1
11719523,6999244,11,1420,1,1119,1621931,43206,NaN,NaN,NaN,NaN,NaN,test,NaN,56,59,44
11719524,6999247,10,1334,1,2008,970421,43069,NaN,NaN,NaN,NaN,NaN,test,2,3,1,1


###  Feature 2:  Calculates de sum of prior weeks Log mean Demands

In [17]:
data['Lags_sum'] = 0
for i in range(2,lag+1):
    data['Lags_sum'] += data['Log_Target_lag%d' %(i)]

In [18]:
data.tail()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,source,Log_Target_lag1,Log_Target_lag2,Log_Target_lag3,Log_Target_lag4,Lags_sum
11719520,6999229,10,4051,1,1106,122768,43202,NaN,NaN,NaN,NaN,NaN,test,6,7,1,4,0
11719521,6999239,11,1427,1,2801,1373090,43307,NaN,NaN,NaN,NaN,NaN,test,NaN,2,8,8,0
11719522,6999242,10,1338,1,1201,789056,1146,NaN,NaN,NaN,NaN,NaN,test,1,1,2,1,0
11719523,6999244,11,1420,1,1119,1621931,43206,NaN,NaN,NaN,NaN,NaN,test,NaN,56,59,44,0
11719524,6999247,10,1334,1,2008,970421,43069,NaN,NaN,NaN,NaN,NaN,test,2,3,1,1,0


###  Feature 3:  Mean Demand per client-product pair - Product/Client demand, Product demand, Global demand.

This is a modification of this kaggle script: https://www.kaggle.com/armalali/grupo-bimbo-inventory-demand/benchmark-medians/comments
I'm changing it to  exp mean of the log, instead of just the mean. It results on better RSMLE. see: 
https://www.kaggle.com/apapiu/grupo-bimbo-inventory-demand/mean-vs-medians-a-mathy-approach

In [19]:
tic()
global_log1p_mean = log1p_mean(train['Demanda_uni_equil'])
prod_log1p_mean = train.groupby('Producto_ID').agg({'Demanda_uni_equil': lambda x: log1p_mean(x)})
prod_client_log1p_mean = train.groupby(['Producto_ID', 'Cliente_ID']).agg({'Demanda_uni_equil':  lambda x: log1p_mean(x)})

prod_log1p_mean_dict = prod_log1p_mean.to_dict()
prod_client_log1p_mean_dict = prod_client_log1p_mean.to_dict()

tac()

Time passed: 0hour:37min:57sec


In [20]:
def gen_log1p_mean_feature(key):
    key = tuple(key)
    product = key[0]
    client = key[1]
    try:
        val = prod_client_log1p_mean_dict['Demanda_uni_equil'][(product,client)]
        
    except:
        try:
            val = prod_log1p_mean_dict['Demanda_uni_equil'][(product)]
        except:
            
            val = global_log1p_mean
            
    return val

In [21]:
print (global_log1p_mean)

3.96747983777


In [22]:
prod_log1p_mean.head()

,Demanda_uni_equil
Producto_ID,
41,82.796566
53,58.309573
72,4.152047
73,2.005731
100,3.324174


In [23]:
prod_client_log1p_mean.head()

Demanda_uni_equil
Producto_ID Cliente_ID                   
41          146030              57.509851
            465617               0.000000
            681747            1505.912233
            684023              54.015921
            685079               0.000000

In [24]:
tic()
data['log1p_mean'] = data[['Producto_ID', 'Cliente_ID']].apply(lambda x:gen_log1p_mean_feature(x), axis=1)
tac()

Time passed: 0hour:3min:5sec


In [25]:
data.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,source,Log_Target_lag1,Log_Target_lag2,Log_Target_lag3,Log_Target_lag4,Lags_sum,log1p_mean
0,NaN,7,1110,7,3301,15766,1216,2,16.760000,0,0,2,train,1,3,2,4,0,2.595359
1,NaN,7,1110,7,3301,15766,1238,2,19.660000,0,0,2,train,2,1,3,4,0,2.306396
2,NaN,7,1110,7,3301,15766,1242,2,15.280000,0,0,2,train,3,2,3,3,0,2.022419
3,NaN,7,1110,7,3301,15766,1250,14,106.959999,0,0,14,train,1,8,3,5,0,5.754628
4,NaN,7,1110,7,3301,15766,1309,9,60.840000,0,0,9,train,6,6,2,3,0,4.134517


### Feature 4: Create a broad category of Brand of item (brand hypothesis)
Let's preprocess products a little bit. I borrowed some of the preprocessing from here: https://www.kaggle.com/vykhand/grupo-bimbo-inventory-demand/exploring-products

In [26]:
products  =  pd.read_csv("input-data/producto_tabla.csv")
products  =  pd.read_csv("input-data/producto_tabla.csv")
#products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=False)#python 2.7
products['short_name'] = products.NombreProducto.str.extract('^(\D*)')#python 3.0
#products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$')
#w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)')
products['weight'] = w[0].astype('float')*w[1].map({'Kg':1000, 'g':1})
#products['pieces'] =  products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')
products['pieces'] =  products.NombreProducto.str.extract('(\d+)p ').astype('float')
products.head()

,Producto_ID,NombreProducto,short_name,brand,weight,pieces
0,0,NO IDENTIFICADO 0,NO IDENTIFICADO,IDENTIFICADO,NaN,NaN
1,9,Capuccino Moka 750g NES 9,Capuccino Moka,NES,750,NaN
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41,Bimbollos Ext sAjonjoli,BIM,480,6
3,53,Burritos Sincro 170g CU LON 53,Burritos Sincro,LON,170,NaN
4,72,Div Tira Mini Doradita 4p 45g TR 72,Div Tira Mini Doradita,TR,45,4


In [27]:
products.tail()

,Producto_ID,NombreProducto,short_name,brand,weight,pieces
2587,49992,Tostado Integral 180g MTA WON 49992,Tostado Integral,WON,180,NaN
2588,49993,Tostado Integral 180g TAB WON 49993,Tostado Integral,WON,180,NaN
2589,49994,Tostado Int 0pct Grasa Azuc 200g WON 49994,Tostado Int,WON,200,NaN
2590,49996,Tostado Int 0pct Grasa Azuc 200g MTA WON 49996,Tostado Int,WON,200,NaN
2591,49997,Tostado Int 0pct Grasa Azuc 200g TAB WON 49997,Tostado Int,WON,200,NaN


In [28]:
products.brand.value_counts()

BIM             679
MLA             657
TR              257
LAR             182
GBI             130
WON             117
DH               95
LON              83
SAN              66
MR               64
ORO              44
CC               33
SL               32
BAR              31
SUA              20
RIC              20
MP               10
SUN               9
JMX               8
SKD               7
MCM               5
COR               5
NAI               4
THO               4
NES               3
TRI               3
CHK               2
KOD               2
BRL               2
MSK               2
PUL               2
NEC               1
MTB               1
AM                1
IDENTIFICADO      1
DIF               1
CAR               1
EMB               1
GV                1
VR                1
LC                1
BRE               1
AV                1
VER               1
Name: brand, dtype: int64

In [29]:
products.brand.nunique()

44

In [30]:
products_id_brand  = products[['Producto_ID', 'brand']].copy()

In [31]:
data = pd.merge(data, products_id_brand, on='Producto_ID')

In [32]:
data.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,source,Log_Target_lag1,Log_Target_lag2,Log_Target_lag3,Log_Target_lag4,Lags_sum,log1p_mean,brand
0,NaN,7,1110,7,3301,15766,1216,2,16.760000,0,0,2,train,1,3,2,4,0,2.595359,BIM
1,NaN,7,1110,7,3301,1682456,1216,2,16.760000,0,0,2,train,1,1,2,1,0,1.379566,BIM
2,NaN,7,1110,7,3301,4603047,1216,3,24.389999,0,0,3,train,3,3,2,5,0,3.310458,BIM
3,NaN,7,1110,7,3302,99974,1216,3,25.139999,0,0,3,train,4,4,4,3,0,3.814964,BIM
4,NaN,7,1110,7,3302,912498,1216,11,92.180000,0,0,11,train,8,6,22,22,0,13.773135,BIM


### Feature 5: Create clusters of Products (utility hypothesis) - ramdonly pick 30 clusters

In [33]:
#Read files:
product_clusters = pd.read_csv('input-data/producto_clusters.csv')

In [34]:
product_clusters.tail()

,Producto_ID,NombreProducto,product_shortname,cluster
2586,49992,Tostado Integral 180g MTA WON 49992,Tostado Integral 180g,4
2587,49993,Tostado Integral 180g TAB WON 49993,Tostado Integral 180g,4
2588,49994,Tostado Int 0pct Grasa Azuc 200g WON 49994,Tostado Int 0pct Grasa Azuc 200g,4
2589,49996,Tostado Int 0pct Grasa Azuc 200g MTA WON 49996,Tostado Int 0pct Grasa Azuc 200g,4
2590,49997,Tostado Int 0pct Grasa Azuc 200g TAB WON 49997,Tostado Int 0pct Grasa Azuc 200g,4


In [35]:
print (product_clusters["cluster"].value_counts())

1     204
14    137
10    136
11    124
4     118
13    109
23    103
24    101
19     99
17     93
16     89
8      88
25     85
30     81
22     78
20     75
15     73
9      73
27     71
5      70
2      68
6      65
3      63
26     62
7      61
28     60
12     60
18     59
29     53
21     33
Name: cluster, dtype: int64


In [36]:
products_id_clusters = product_clusters[['Producto_ID', 'cluster']].copy()

In [37]:
products_id_clusters.tail()

,Producto_ID,cluster
2586,49992,4
2587,49993,4
2588,49994,4
2589,49996,4
2590,49997,4


In [38]:
data = pd.merge(data, products_id_clusters, on='Producto_ID')

In [39]:
data.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,...,Demanda_uni_equil,source,Log_Target_lag1,Log_Target_lag2,Log_Target_lag3,Log_Target_lag4,Lags_sum,log1p_mean,brand,cluster
0,NaN,7,1110,7,3301,15766,1216,2,16.760000,0,...,2,train,1,3,2,4,0,2.595359,BIM,2
1,NaN,7,1110,7,3301,1682456,1216,2,16.760000,0,...,2,train,1,1,2,1,0,1.379566,BIM,2
2,NaN,7,1110,7,3301,4603047,1216,3,24.389999,0,...,3,train,3,3,2,5,0,3.310458,BIM,2
3,NaN,7,1110,7,3302,99974,1216,3,25.139999,0,...,3,train,4,4,4,3,0,3.814964,BIM,2
4,NaN,7,1110,7,3302,912498,1216,11,92.180000,0,...,11,train,8,6,22,22,0,13.773135,BIM,2


### Feature 6: Create a category of Size of store based on Number of Agencies and Routes and Sales Channels that serve the store

In [40]:
#Determine pivot table
Rutas_per_store = data.pivot_table(values=["Ruta_SAK"], index=["Cliente_ID"], aggfunc=pd.Series.nunique)
Rutas_per_store.head()

,Ruta_SAK
Cliente_ID,
26,3
60,13
65,1
101,1
105,1


In [41]:
Rutas_per_store.describe()

,Ruta_SAK
count,572005.000000
mean,1.884597
std,1.145583
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,46.000000


In [42]:
Agencies_per_store = data.pivot_table(values=["Agencia_ID"], index=["Cliente_ID"], aggfunc=pd.Series.nunique)
Agencies_per_store.head()

,Agencia_ID
Cliente_ID,
26,2
60,2
65,1
101,1
105,1


In [43]:
Agencies_per_store.describe()

,Agencia_ID
count,572005.000000
mean,1.034028
std,0.207211
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,62.000000


In [44]:
Canals_per_store = data.pivot_table(values=["Canal_ID"], index=["Cliente_ID"], aggfunc=pd.Series.nunique)
Canals_per_store.head()

,Canal_ID
Cliente_ID,
26,1
60,1
65,1
101,1
105,1


In [45]:
Canals_per_store.describe()

,Canal_ID
count,572005.000000
mean,1.002661
std,0.051920
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,4.000000


It doesn't look that we can Bin on Canal_ID or Agencia_ID, but it does look like Ruta_SAK is a good option!"

In [46]:
Rutas_per_store.rename(columns={'Ruta_SAK': 'Qty_Ruta_SAK'}, inplace=True)

In [47]:
Rutas_per_store.head()

,Qty_Ruta_SAK
Cliente_ID,
26,3
60,13
65,1
101,1
105,1


In [48]:
#Mergin Routa_Sak's per client to data df
data = pd.merge(data,Rutas_per_store,right_index=True, left_on='Cliente_ID')

In [49]:
data.tail()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,...,source,Log_Target_lag1,Log_Target_lag2,Log_Target_lag3,Log_Target_lag4,Lags_sum,log1p_mean,brand,cluster,Qty_Ruta_SAK
11719282,6199686,11,2263,1,2101,4562421,35307,NaN,NaN,NaN,...,test,NaN,3,2,2,0,2.301927,MLA,24,1
11719305,1810966,11,1682,11,5708,438562,40931,NaN,NaN,NaN,...,test,NaN,17,16,15,0,15.980369,MLA,7,1
11719352,1066314,11,23899,1,2902,2090985,37360,NaN,NaN,NaN,...,test,NaN,5,3,3,0,3.578857,MLA,26,1
11719360,1472254,11,3229,6,3203,156451,37360,NaN,NaN,NaN,...,test,NaN,33,7,10,0,13.409664,MLA,26,1
11719361,1580430,11,1279,7,3904,980308,37360,NaN,NaN,NaN,...,test,NaN,10,10,10,0,10.000000,MLA,26,1


In [50]:
#Binning:
def binning(col, cut_points, labels=None):
  #Define min and max values:
  minval = col.min()
  maxval = col.max()

  #create list by adding min and max to cut_points
  break_points = [minval] + cut_points + [maxval]

  #if no labels provided, use default labels 0 ... (n-1)
  if not labels:
    labels = range(len(cut_points)+1)

  #Binning using cut function of pandas
  colBin = pd.cut(col,bins=break_points,labels=labels,include_lowest=True)
  return colBin

#Binning Qty_Ruta_SAK:
cut_points = [2,4,10]
labels = ["low","medium","high","very high"]
data["Qty_Ruta_SAK_Bin"] = binning(data["Qty_Ruta_SAK"], cut_points, labels)
print (pd.value_counts(data["Qty_Ruta_SAK_Bin"], sort=False))

low          5438414
medium       4617147
high         1593450
very high      70514
dtype: int64


In [51]:
#We don't need Qty_Ruta_Sak anymore
data.drop(['Qty_Ruta_SAK'],axis=1,inplace=True)

In [52]:
data.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,...,source,Log_Target_lag1,Log_Target_lag2,Log_Target_lag3,Log_Target_lag4,Lags_sum,log1p_mean,brand,cluster,Qty_Ruta_SAK_Bin
0,NaN,7,1110,7,3301,15766,1216,2,16.760000,0,...,train,1,3,2,4,0,2.595359,BIM,2,low
49030,NaN,8,1110,7,3301,15766,1216,5,41.900002,0,...,train,2,1,3,2,0,2.595359,BIM,2,low
155343,NaN,7,1110,7,3301,15766,1238,2,19.660000,0,...,train,2,1,3,4,0,2.306396,BIM,2,low
216351,NaN,8,1110,7,3301,15766,1238,3,29.490000,0,...,train,2,2,1,3,0,2.306396,BIM,2,low
273406,NaN,9,1110,7,3301,15766,1238,2,19.660000,0,...,train,3,2,2,1,0,2.306396,BIM,2,low


### Feature 7: Create a category of location based on zip code (embedded on town table)

In [53]:
import re 
import os
import time
towns = pd.read_csv("input-data/town_state.csv")
L = lambda x: list(map(int, re.findall('\d+', x)))[0]
towns['ZipCode'] = towns.Town.apply(L) 
towns['ZipCode'] = np.uint16(towns['ZipCode'])

In [54]:
zipcodes_df = towns[['Agencia_ID', 'ZipCode']].copy()

In [55]:
zipcodes_df.head()

,Agencia_ID,ZipCode
0,1110,2008
1,1111,2002
2,1112,2004
3,1113,2008
4,1114,2029


In [56]:
data = pd.merge(data, zipcodes_df, on='Agencia_ID')

In [57]:
data.tail()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,...,Log_Target_lag1,Log_Target_lag2,Log_Target_lag3,Log_Target_lag4,Lags_sum,log1p_mean,brand,cluster,Qty_Ruta_SAK_Bin,ZipCode
11719520,NaN,9,1961,7,3202,4525143,46215,180,1728.0,0,...,168,82,138,114,0,90.597525,SL,17,low,2297
11719521,3112739,10,1961,7,3202,4525143,46215,NaN,NaN,NaN,...,180,168,82,138,0,90.597525,SL,17,low,2297
11719522,NaN,7,1961,7,3202,4610118,46149,8,166.0,0,...,6,6,6,14,0,8.350862,BIM,16,low,2297
11719523,NaN,8,1961,7,3202,4610118,46149,8,166.0,0,...,8,6,6,6,0,8.350862,BIM,16,low,2297
11719524,NaN,9,1961,7,3202,4610118,46149,14,290.5,0,...,8,8,6,6,0,8.350862,BIM,16,low,2297


In [58]:
data.apply(lambda x: len(x.unique()))

id                   1466486
Semana                     5
Agencia_ID               552
Canal_ID                   9
Ruta_SAK                2557
Cliente_ID            572005
Producto_ID             1359
Venta_uni_hoy           1575
Venta_hoy              47163
Dev_uni_proxima          337
Dev_proxima             7487
Demanda_uni_equil       1581
source                     2
Log_Target_lag1         4575
Log_Target_lag2         5068
Log_Target_lag3         5074
Log_Target_lag4         5069
Lags_sum                   1
log1p_mean            603655
brand                     26
cluster                   30
Qty_Ruta_SAK_Bin           4
ZipCode                  254
dtype: int64

### Numerical and One-Hot Coding of Categorical variables
Since scikit-learn accepts only numerical variables, so i have to convert all categories of nominal variables into numeric types.

Lets start with coding all low cardinality nominal categorical variables (brand, Qty_Ruta_SAK_Bin)  as numeric using ‘LabelEncoder’ from sklearn’s preprocessing module.

In [59]:
print (data.dtypes)

id                   float64
Semana                  int8
Agencia_ID            uint16
Canal_ID                int8
Ruta_SAK               int32
Cliente_ID             int32
Producto_ID            int32
Venta_uni_hoy        float64
Venta_hoy            float64
Dev_uni_proxima      float64
Dev_proxima          float64
Demanda_uni_equil    float64
source                object
Log_Target_lag1      float64
Log_Target_lag2      float64
Log_Target_lag3      float64
Log_Target_lag4      float64
Lags_sum             float64
log1p_mean           float64
brand                 object
cluster                int64
Qty_Ruta_SAK_Bin      object
ZipCode               uint16
dtype: object


In [60]:
#Import library:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

var_mod = ['brand', 'Qty_Ruta_SAK_Bin']
for i in var_mod:
    data[i] = le.fit_transform(data[i])

In [61]:
data.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,...,Log_Target_lag1,Log_Target_lag2,Log_Target_lag3,Log_Target_lag4,Lags_sum,log1p_mean,brand,cluster,Qty_Ruta_SAK_Bin,ZipCode
0,NaN,7,1110,7,3301,15766,1216,2,16.760000,0,...,1,3,2,4,0,2.595359,1,2,1,2008
1,NaN,8,1110,7,3301,15766,1216,5,41.900002,0,...,2,1,3,2,0,2.595359,1,2,1,2008
2,NaN,7,1110,7,3301,15766,1238,2,19.660000,0,...,2,1,3,4,0,2.306396,1,2,1,2008
3,NaN,8,1110,7,3301,15766,1238,3,29.490000,0,...,2,2,1,3,0,2.306396,1,2,1,2008
4,NaN,9,1110,7,3301,15766,1238,2,19.660000,0,...,3,2,2,1,0,2.306396,1,2,1,2008


One-Hot-Coding refers to creating dummy variables, one for each category of a categorical variable. For example, the 'cluster' variable has 30 categories. One hot coding will remove this variable and generate 30 new variables. Each will have binary numbers – 0 (if the category is not present) and 1(if category is present).
Categorical variables are intentionally (for censorship) or implicitly encoded as numerical variables in order to be used as features in any given model.

e.g. [house, car, tooth, car] becomes [0,1,2,1].

This imparts an ordinal property to the variable, i.e. house < car < tooth.

As this is ordinal characteristic is usually not desired, one hot encoding is necessary for the proper representation of the distinct elements of the variable.

-- This can be done using ‘get_dummies’ function of Pandas.


In [62]:
#One Hot Coding: you need python 3 and 128GB ram for this
#tic()
#data = pd.get_dummies(data, columns=['Canal_ID','brand','cluster','Qty_Ruta_SAK_Bin'])
#tac()

Lets look at the datatypes of columns now:

In [63]:
data.dtypes

id                   float64
Semana                  int8
Agencia_ID            uint16
Canal_ID                int8
Ruta_SAK               int32
Cliente_ID             int32
Producto_ID            int32
Venta_uni_hoy        float64
Venta_hoy            float64
Dev_uni_proxima      float64
Dev_proxima          float64
Demanda_uni_equil    float64
source                object
Log_Target_lag1      float64
Log_Target_lag2      float64
Log_Target_lag3      float64
Log_Target_lag4      float64
Lags_sum             float64
log1p_mean           float64
brand                  int64
cluster                int64
Qty_Ruta_SAK_Bin       int64
ZipCode               uint16
dtype: object

## 5\. Exporting Data

In [64]:
#Divide into test and train:
import csv
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

#Drop unnecessary columns:
train.drop(['source','id','Venta_uni_hoy','Venta_hoy','Dev_uni_proxima','Dev_proxima'],axis=1,inplace=True)
test.drop(['source','Venta_uni_hoy','Venta_hoy','Dev_uni_proxima', 'Dev_proxima','Demanda_uni_equil'],axis=1,inplace=True)

#Export files as modified versions:
tic()
train.to_csv("./input-data/train_modified.csv", index=False, quoting=csv.QUOTE_NONE)
test.to_csv("./input-data/test_modified.csv", index=False, quoting=csv.QUOTE_NONE)
tac()

C:\Anaconda\envs\py35\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\envs\py35\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Time passed: 0hour:2min:37sec
